In [107]:
import langchain_community 
import langchain
import langchain_openai

In [108]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<api-key>'

In [110]:
os.environ['OPENAI_API_KEY'] = '<api-key>'

In [111]:
import json

# Load the JSON data
with open("../../monsters_response.json", "r") as file:
    json_data = json.load(file)

# Extract the monsters list
monsters = json_data["monsters"]

print(monsters)

[{'monster_id': 'deathclaw_1', 'lon': 85.5487, 'lat': 17.9072}, {'monster_id': 'deathclaw_2', 'lon': 87.3032, 'lat': 19.4866}, {'monster_id': 'super_mutant_1', 'lon': 83.1134, 'lat': 19.163}, {'monster_id': 'super_mutant_2', 'lon': 82.8223, 'lat': 18.762}, {'monster_id': 'super_mutant_3', 'lon': 87.0667, 'lat': 20.6589}, {'monster_id': 'super_mutant_4', 'lon': 83.4131, 'lat': 22.1663}, {'monster_id': 'super_mutant_5', 'lon': 81.6439, 'lat': 18.7261}, {'monster_id': 'radscorpion_1', 'lon': 87.2045, 'lat': 18.7888}, {'monster_id': 'radscorpion_2', 'lon': 86.0467, 'lat': 21.2248}, {'monster_id': 'radscorpion_3', 'lon': 86.5529, 'lat': 21.1394}, {'monster_id': 'radscorpion_4', 'lon': 83.6146, 'lat': 20.4416}, {'monster_id': 'yao_guai_1', 'lon': 85.4507, 'lat': 18.2524}, {'monster_id': 'yao_guai_2', 'lon': 83.6133, 'lat': 21.7572}, {'monster_id': 'yao_guai_3', 'lon': 86.3814, 'lat': 20.8939}, {'monster_id': 'feral_ghoul_1', 'lon': 85.8137, 'lat': 21.4151}, {'monster_id': 'feral_ghoul_2', 'l

In [112]:
from langchain_core.documents import Document

# Convert monsters to LangChain Document objects
docs = [
    Document(
        page_content=f"Monster ID: {monster['monster_id']}, Location: ({monster['lat']}, {monster['lon']})",
        metadata={"monster_id": monster["monster_id"], "latitude": monster["lat"], "longitude": monster["lon"]}
    )
    for monster in monsters
]

# Print the documents
for doc in docs:
    print(doc)

page_content='Monster ID: deathclaw_1, Location: (17.9072, 85.5487)' metadata={'monster_id': 'deathclaw_1', 'latitude': 17.9072, 'longitude': 85.5487}
page_content='Monster ID: deathclaw_2, Location: (19.4866, 87.3032)' metadata={'monster_id': 'deathclaw_2', 'latitude': 19.4866, 'longitude': 87.3032}
page_content='Monster ID: super_mutant_1, Location: (19.163, 83.1134)' metadata={'monster_id': 'super_mutant_1', 'latitude': 19.163, 'longitude': 83.1134}
page_content='Monster ID: super_mutant_2, Location: (18.762, 82.8223)' metadata={'monster_id': 'super_mutant_2', 'latitude': 18.762, 'longitude': 82.8223}
page_content='Monster ID: super_mutant_3, Location: (20.6589, 87.0667)' metadata={'monster_id': 'super_mutant_3', 'latitude': 20.6589, 'longitude': 87.0667}
page_content='Monster ID: super_mutant_4, Location: (22.1663, 83.4131)' metadata={'monster_id': 'super_mutant_4', 'latitude': 22.1663, 'longitude': 83.4131}
page_content='Monster ID: super_mutant_5, Location: (18.7261, 81.6439)' me

In [113]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# Split the documents
split_docs = text_splitter.split_documents(docs)

print(split_docs)

[Document(metadata={'monster_id': 'deathclaw_1', 'latitude': 17.9072, 'longitude': 85.5487}, page_content='Monster ID: deathclaw_1, Location: (17.9072, 85.5487)'), Document(metadata={'monster_id': 'deathclaw_2', 'latitude': 19.4866, 'longitude': 87.3032}, page_content='Monster ID: deathclaw_2, Location: (19.4866, 87.3032)'), Document(metadata={'monster_id': 'super_mutant_1', 'latitude': 19.163, 'longitude': 83.1134}, page_content='Monster ID: super_mutant_1, Location: (19.163, 83.1134)'), Document(metadata={'monster_id': 'super_mutant_2', 'latitude': 18.762, 'longitude': 82.8223}, page_content='Monster ID: super_mutant_2, Location: (18.762, 82.8223)'), Document(metadata={'monster_id': 'super_mutant_3', 'latitude': 20.6589, 'longitude': 87.0667}, page_content='Monster ID: super_mutant_3, Location: (20.6589, 87.0667)'), Document(metadata={'monster_id': 'super_mutant_4', 'latitude': 22.1663, 'longitude': 83.4131}, page_content='Monster ID: super_mutant_4, Location: (22.1663, 83.4131)'), D

In [119]:
%pip install sentence-transformers

  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
Using cached sentence_transformers-3.3.1-py3-none-any.whl (268 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\KIIT\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\KIIT\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [118]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import json
import time

# Load your monsters JSON data
with open("../../monsters_response.json", "r") as file:
    data = json.load(file)

# Prepare documents for embedding
split_docs = []
for monster in data.get("monsters", []):
    content = f"Monster ID: {monster['monster_id']}, Location: ({monster['lon']}, {monster['lat']})"
    split_docs.append(Document(page_content=content, metadata={"monster_id": monster["monster_id"]}))

print(f"Total documents to process: {len(split_docs)}")

# Initialize SentenceTransformer model (Hugging Face model)
model = SentenceTransformer('all-MiniLM-L6-v2')  # This is a popular lightweight model for embeddings

# Define batch size
BATCH_SIZE = 10
all_vectors = []
all_metadatas = []

# Process documents in batches
for i in range(0, len(split_docs), BATCH_SIZE):
    batch = split_docs[i:i + BATCH_SIZE]
    print(f"Processing batch {i // BATCH_SIZE + 1}: {len(batch)} documents")

    try:
        # Extract content and metadata
        batch_texts = [doc.page_content for doc in batch]
        batch_metadatas = [doc.metadata for doc in batch]

        # Generate embeddings for the current batch
        batch_vectors = model.encode(batch_texts)  # Generate embeddings using SentenceTransformer

        # Append to the global lists
        all_vectors.extend(batch_vectors)
        all_metadatas.extend(batch_metadatas)

        # Optional: Add delay to avoid hitting rate limits
        time.sleep(1)  # Adjust as needed

    except Exception as e:
        print(f"Error processing batch {i // BATCH_SIZE + 1}: {e}")

# Store embeddings and metadata in FAISS
print("Storing vectors in FAISS...")
vector_store = FAISS.from_embeddings(all_vectors, all_metadatas)

# Save vector store locally
vector_store.save_local("faiss_index")
print("FAISS vector store saved successfully.")

ModuleNotFoundError: No module named 'sentence_transformers'